<a href="https://colab.research.google.com/github/eakkasitgithub/hightopo/blob/master/frame_capture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install required external modules**
* opencv-python-headless 4.5.5.52 caused a [registermattype-from-cv2-cv2] error on Google Colab --> downgraded to the latest workable version 4.1.2.30

In [ ]:
!pip install easyocr
!pip uninstall -y opencv-python-headless
!pip install opencv-python-headless==4.1.2.30

Main Functions Cell

In [ ]:
# Google Colab Default Modules
import time, sys, cv2, threading
import pandas as pd
import numpy as np
import queue as Queue
import json

from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from scipy.signal import find_peaks

# External Module
import easyocr

# Buffers-less VideoCapture
class VideoCapture:

    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.cap.set(cv2.CAP_PROP_FPS, 30)
        self.q = Queue.Queue()
        t = threading.Thread(target=self._reader)
        t.daemon = True
        t.start()

    # read frames as soon as they are available, keeping only most recent one
    def _reader(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            if not self.q.empty():
                try:
                    self.q.get_nowait()  # discard previous (unprocessed) frame
                except Queue.Empty:
                    pass
            self.q.put(frame)

    def read(self):
        return self.q.get()


# Initiate Jaided AI/Easy OCR
reader = easyocr.Reader(['en'])

def easyocr_engine_reader(img, canvas, cy0, cx0):
    # Jaided AI/Easy OCR
    # Reader.readtext() includes img pre-processing pipeline to optimize img for the OCR engine, tests were done -- OpenCV img preprocessing not beneficially improve results compared to increase processing time
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    result = reader.readtext(img, allowlist='0123456789')

    # Assign extracted result to returning list
    extracted_numeric = 'nan'
    for word in result:
        extracted_numeric = word[1]

    '''
    --- Visualize result on canvas EasyOCR result structure
    = [ [ [[x1, y1], [x2,y1], [x1, y2], [x2, y2]], text, confident level ], [ [...], text, cl], [ ......] ]
    where x1, y1, x2, y2 are the bounding coordinate of the text
          cy0, cx0 are the coordinate of cropped img relative to the full img
    for word in result:
        x1, y1, x2, y2 = int(word[0][0][0]), int(word[0][0][1]), int(word[0][2][0]), int(word[0][2][1])
        cv2.rectangle(canvas, (cx0 + x1, cy0 + y1), (cx0 + x2, cy0 + y2), (255, 255, 0), 1)
        cv2.putText(canvas, word[1], (cx0 + x1, cy0 + y1 + 12), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 0), 1)
    '''
    return extracted_numeric, canvas


def CheckSignal(img):
  valid = False
  accum, count = '', 0
  
  resY, resX = img.shape[:2]
  scaleY, scaleX = np.divide([resY, resX], [720, 1280])
  scale = scaleY**2/scaleX

  if scaleY == scaleX:
    if np.all(img == 0):
      accum = 'Blank frame detected.'
    else:
      y0, y1, x0, x1 = [int(i*scale) for i in [55, 83, 125, 300]]
      signalframe = img[y0:y1, x0:x1, :]
      signalframe = cv2.cvtColor(signalframe, cv2.COLOR_BGR2GRAY)
      result = reader.readtext(signalframe, allowlist='')
      for word in result:
        accum += word[1] + ' '
        for char in word[1]:
          count += int(char.lower() in 'nosignal')

      if (accum.lower() == 'nosignal') or (count > 7):
        accum = accum + 'was read from the monitor.'
      else:
        valid = True
  else:
    accum = 'Distorted frame ratio.'

  if valid == False:
    print('Check signal: ', accum, 'input resolution: ', img.shape[:2], ' vid_ratio: ', img.shape[1]/img.shape[0], 'valid_scale:', scale)
  return scale, valid


def ExtractGraphLine(img, key):
    h, w = img.shape[:2]
    mask = np.zeros((h, w), np.uint8)

    # Transform to gray colorspace
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Conditional image processing for each type of graph
    if key in ['ECG']:
        kernel = np.ones((2, 2), np.uint8)
        gray = cv2.erode(gray, kernel, iterations=1)
        gray = cv2.dilate(gray, (1,1), iterations=1)
    if key in ['ART']:
        gray[gray < (np.max(gray) * 0.45)] = 0

    # threshold the image
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Perform opening on the thresholded image (erosion followed by dilation)
    kernel = np.ones((2, 2), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Search for contours and select the biggest one and draw it on mask
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cnt = max(contours, key=cv2.contourArea)
    cv2.drawContours(mask, [cnt], 0, 255, -1)
    mask = cv2.dilate(mask, kernel, iterations=1)

    # Perform a bitwise operation
    res = cv2.bitwise_not(mask)

    return res


def TemporalSection(value):
    TMavg = 3
    SectionIndexArray = []
    samples = value.shape[0]

    # Peak Detection of Moving Variance Method
    MovingVariance = np.zeros((samples - TMavg))
    for index in range(samples - TMavg):
        MovingVariance[index] = np.abs(np.var(value[index:index + TMavg]))
    peaks, _ = find_peaks(MovingVariance, prominence=1)

    if len(peaks):
        if peaks[0] != 0:
            peaks = np.insert(peaks, 0, 0)
        if peaks[-1] != (samples - 1):
            peaks = np.append(peaks, samples - 1)
        SectionIndexArray = [[peaks[i], peaks[i + 1]] for i in range(len(peaks)-1)]

    return SectionIndexArray


def ReadGraph(img, x0, y0, key, zero_level, scale_h, scale_value, mask):
    B, G, R = img[:, :, 0], img[:, :, 1], img[:, :, 2]

    for each in mask:
        if each:
            xroi0, xroi1, yroi0, yroi1 = each
            xrel0 = max(0, xroi0 - x0)
            xrel1 = max(0, xroi1 - x0)
            yrel0 = max(0, yroi0 - y0)
            yrel1 = max(0, yroi1 - y0)
            ROI = img[yrel0:yrel1, xrel0:xrel1, :]
            mask_gray = cv2.cvtColor(ROI, cv2.COLOR_BGR2GRAY)
            _, bin_mask = cv2.threshold(mask_gray, 10, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            mask = cv2.bitwise_not(bin_mask)
            masked_ROI = cv2.bitwise_or(ROI, ROI, mask=mask)
            # cv2.imshow(key + 'img_masked', mask)
            img[yrel0:yrel1, xrel0:xrel1, :] = masked_ROI

    # Conditional image processing for each type of graph
    if key in ['ECG', 'Pleth']:
        # Remove white notation from image
        mask_white = (B >= G) & (R >= G)
        img[np.where(mask_white == 1)] = [0, 0, 0]

    if key in ['ART']:
        mask_level = (R > 120)
        img[np.where(mask_level == 0)] = [0, 0, 0]

    crop_img_gray = ExtractGraphLine(img,key)
    index = np.array([np.nanargmax(crop_img_gray[:, col]) for col in range(np.shape(crop_img_gray)[1])])

    # Scaling Abstract Value to Related Dimensional Units
    frame_h, frame_w = img.shape[:2]
    value = frame_h - index
    value = ((value - zero_level) * scale_value) / scale_h
    extracted_value = value

    # Visualize graphical recreation of extracted value
    canvas = np.zeros((img.shape[0], img.shape[1]), np.uint8)
    for col in range(frame_w):
        i = index[col]
        if i != 0:
            try:
                if not (previous_range in range(i - 1, i + 1)):
                    if previous_range < i:
                        canvas[previous_range:i, col] = 255
                    else:
                        canvas[i:previous_range, col] = 255
                else:
                    canvas[i, col] = 255
            except:
                # print(i, 'invalid i value')
                pass
            previous_range = i

    interval = TemporalSection(extracted_value)
    extracted_value = [extracted_value[interval[i][0]: interval[i][1]].tolist() for i in range(len(interval))]

    return extracted_value, canvas

# Define storing parameters and biosignals of interest 
extracted_biosignals, f_crop = {}, {}
biosignals = {'HR': {'fLoc': [78, 77 + 94, 56, 56 + 237], 'dtype': 'num'}, 
 'RR': {'fLoc': [599, 599 + 63, 200, 200 + 94], 'dtype': 'num'}, 
 'N_SBP': {'fLoc': [223, 223 + 48, 57, 57 + 109], 'dtype': 'num'}, 
 'N_DBP': {'fLoc': [222, 222 + 47, 174, 174 + 112], 'dtype': 'num'}, 
 'N_MAP': {'fLoc': [269, 269 + 36, 133, 133 + 73], 'dtype': 'num'},
 'A_SBP': {'fLoc': [343, 343 + 48, 63, 63 + 109], 'dtype': 'num'}, 
 'A_DBP': {'fLoc': [344, 344 + 46, 179, 179 + 112], 'dtype': 'num'}, 
 'A_MAP': {'fLoc': [389, 389 + 36, 143, 143 + 73], 'dtype': 'num'}, 
 'SpO2': {'fLoc': [492, 492 + 79, 162, 162 + 132], 'dtype': 'num'},
 'ECG': {'fLoc': [64, 240, 503, 1280], 'dtype': 'gr', 'label': [[176, 39], 1, 'mV'], 'mask': [[]]}, 
 'ART': {'fLoc': [322, 433, 503, 1280], 'dtype': 'gr', 'label': [[432, 111], 200, 'mmHg'], 'mask': [[1180, 1180 + 29, 322, 322 + 16], [1241, 1241 + 24, 322, 322 + 12], [1242, 1242 + 8, 424, 424 + 9]]}, 
 'Pleth': {'fLoc': [470, 584, 503, 1280], 'dtype': 'gr', 'label': [[584, 584 - 476], 1, ''], 'mask': [[1252, 1252 + 17, 472, 472 + 15], [1245, 1245 + 31, 492, 492 + 14]]}, 
 'Resp': {'fLoc': [584, 713, 503, 1280], 'dtype': 'gr', 'label': [[713, 713 - 584], 1, '%'], 'mask': [[1252, 1252 + 16, 585, 585 + 14]]}}
# -- For web streaming capture -- Working only these four channels at the moment
#VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-00"
VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-04"
#VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-06"
#VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-12"

stream_capture = VideoCapture(VIDEO_URL)

if (stream_capture.cap.isOpened() == False):
    print('!!! Unable to open URL')
    sys.exit(-1)

while(True):
    # -- Read one frame
    frame = stream_capture.read()
    #path = '/content/drive/MyDrive/Work/med_cmu_project/icu_surg/streaming/signal1.png'
    #frame = cv2.imread(path)
    t_cap = str(time.ctime())
    
    # -- Use the frame object to extract numbers and time series data from here

    # Visualize Captured Frame
    '''
    clear_output()  
    cv2_imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) 
    '''

    # Check signal validity by calling function 'CheckSignal'
    scale, valid = CheckSignal(frame)
    if valid:

      # Extract numeric and graphical value into DataFrame (df_output)
      for key in biosignals.keys():
        y0, y1, x0, x1 = biosignals[key]['fLoc'][0], biosignals[key]['fLoc'][1], biosignals[key]['fLoc'][2], biosignals[key]['fLoc'][3]
        y0, y1, x0, x1 = [ int(i*scale) for i in [y0, y1] + [x0, x1]]
        f_crop[key] = frame[y0: y1, x0: x1, :]
        cv2_imshow(cv2.cvtColor(f_crop[key], cv2.COLOR_BGR2RGB)) 
        datatype = biosignals[key]['dtype']
        if datatype == 'gr':
          label = biosignals[key]['label']
          rel0 = y1 - int(scale*label[0][0])
          scale_h, scale_value = int(scale*label[0][1]), label[1]
          # OGR by calling series of nested functions 'ReadGraph'
          extracted_biosignals[key], graphic_canvas = ReadGraph(f_crop[key], x0, y0, key, rel0, scale_h, scale_value, biosignals[key]['mask'])

        elif datatype == 'num':
          # OCR by calling the function 'easyocr_enginer_reader'
          extracted_biosignals[key], ocr_canvas = easyocr_engine_reader(f_crop[key], frame, y0, x0)
        
        else:
          extracted_biosignals[key] = 'data type is not determined.'
        
      # Covert Dict --> JSON for the Firebase storage  
      extracted_biosignals['timestamp'] = t_cap
      json_output = json.dumps(extracted_biosignals)
      print(json_output)
      
      # Save frame.jpg with RGB profile
      cv2.imwrite(t_cap + ".jpg", cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    else:
      print('The frame was not processed, no JSON output created on', t_cap)
      
    # Processing interval (s)
    time.sleep(5)

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Google Colab Default Modules
import time, sys, cv2, threading
import pandas as pd
import numpy as np
import queue as Queue
import json

from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from scipy.signal import find_peaks
from operator import itemgetter
import math

# External Module
import easyocr

class VideoCapture:

    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.cap.set(cv2.CAP_PROP_FPS, 30)
        self.q = Queue.Queue()
        t = threading.Thread(target=self._reader)
        t.daemon = True
        t.start()

    # read frames as soon as they are available, keeping only most recent one
    def _reader(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            if not self.q.empty():
                try:
                    self.q.get_nowait()  # discard previous (unprocessed) frame
                except Queue.Empty:
                    pass
            self.q.put(frame)

    def read(self):
        return self.q.get()

def findPosition(frame,template):
  path = '/content/drive/MyDrive/Work/med_cmu_project/icu_surg/streaming/'+template+'.png'
  img1 = cv2.imread(path)
  gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

  res = cv2.matchTemplate(frame,gray,cv2.TM_CCOEFF_NORMED)
  min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

  #w, h = gray.shape[::-1]
  #top_left = max_loc
  #bottom_right = (top_left[0] + w, top_left[1] + h)
  return max_loc,max_val

def ocr_reader(img):
  # Jaided AI/Easy OCR
  # Reader.readtext() includes img pre-processing pipeline to optimize img for the OCR engine, tests were done -- OpenCV img preprocessing not beneficially improve results compared to increase processing time
  #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return reader.readtext(img, allowlist='0123456789/()-?')

# Initiate Jaided AI/Easy OCR
reader = easyocr.Reader(['en'])

# -- For web streaming capture -- Working only these four channels at the moment
VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-00"
#VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-04"
#VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-06"
#VIDEO_URL = "rtmp://110.238.114.246:1935/live/icu-12"

stream_capture = VideoCapture(VIDEO_URL)

if (stream_capture.cap.isOpened() == False):
    print('!!! Unable to open URL')
    sys.exit(-1)

templates = ['hr','spo2','art','pap','cvp','rr']
left = 1380
while(True):
    # -- Read one frame
    frame = stream_capture.read()
    #cv2_imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    #path = '/content/drive/MyDrive/Work/med_cmu_project/icu_surg/streaming/signal1.png'
    #frame = cv2.imread(path)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gap = 10
    for i in templates:
      top_left,prop = findPosition(gray,i)
      #cv2.rectangle(frame,(left,top_left[1]), (left+300,top_left[1]+120), 255, 2)
      img = gray[top_left[1]:top_left[1]+120,left:left+300]
      thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
      hh, ww = thresh.shape
      white = np.where(thresh==255)
      xmin, ymin, xmax, ymax = np.min(white[1]), np.min(white[0]), np.max(white[1]), np.max(white[0])

      cv2.rectangle(frame,(left+xmin-gap,top_left[1]+ymin-gap), (left+xmax+gap,top_left[1]+ymax+gap), 255, 2)
      print(i)
      if prop > 0.9:
        result = ocr_reader(gray[top_left[1]+ymin-gap:top_left[1]+ymax+gap,left+xmin-gap:left+xmax+gap])
        #Get the best result
        result = max(result,key=itemgetter(2))
        print(result)
      else:
        print('No matched signal')
    cv2_imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) 
    break

stream_capture.cap.release()

Read data from firebase database

In [ ]:
pip install firebase_admin

In [ ]:
import firebase_admin
from firebase_admin import db
from firebase_admin import firestore
from datetime import datetime
import cv2
from google.colab.patches import cv2_imshow
import lzma
import numpy as np

import zlib

path = '/content/drive/MyDrive/Work/med_cmu_project/icu_surg/icusignal-firebase-adminsdk-um0pn-83649d5ef3.json'

if not firebase_admin._apps:
	cred_obj = firebase_admin.credentials.Certificate(path)
	default_app = firebase_admin.initialize_app(cred_obj)

db = firestore.client()
'''
doc_ref = db.collection('icusurg-data').document('bed-01')
t = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

doc_ref.set({
    'o2':{t: 98}
}, merge=True)

emp_ref = db.collection('icusurg-data')
docs = emp_ref.stream()

for doc in docs:
    print('{} => {} '.format(doc.id, doc.to_dict()))
'''

doc_ref = db.collection('icusurg-frame').document('bed-01')

path = '/content/drive/MyDrive/Work/med_cmu_project/icu_surg/streaming/signal1.png'
frame = cv2.imread(path)
print(frame.shape)
#compressor = lzma.LZMACompressor(format=lzma.FORMAT_RAW, filters=[{"id": lzma.FILTER_LZMA2}])
#s = lzma.compress(frame)
s = zlib.compress(frame, zlib.Z_BEST_COMPRESSION)
print(frame.flatten()[6000000:6000100])
print(len(s))

#t = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#doc_ref.set({'timestamp':t, 'shape':list(frame.shape), 'image':s}, merge=False)

'''
docs = db.collection('icusurg-frame').stream()
for doc in docs:
  d = doc.to_dict()
  print(d['image'])
  data = lzma.decompress(d['image'])
  data = np.frombuffer(data, dtype=np.uint8)
  data = data.reshape((*d['shape'], ))
  cv2_imshow(cv2.cvtColor(data, cv2.COLOR_BGR2RGB)) 
'''
'''
path = '/content/drive/MyDrive/Work/med_cmu_project/icu_surg/streaming/signal1.png'
frame = cv2.imread(path)
print(frame.shape)
s = lzma.compress(frame)
print(s)
'''

In [ ]:
#Delete collection
import firebase_admin
from firebase_admin import db
from firebase_admin import firestore

path = '/content/drive/MyDrive/Work/med_cmu_project/icu_surg/icusignal-firebase-adminsdk-um0pn-83649d5ef3.json'

if not firebase_admin._apps:
	cred_obj = firebase_admin.credentials.Certificate(path)
	default_app = firebase_admin.initialize_app(cred_obj)

db = firestore.client()

def delete_collection(coll_ref, batch_size):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

#delete_collection(db.collection('icusurg-data'), 100)

emp_ref = db.collection('icusurg-data')
docs = emp_ref.stream()
n = 1
for doc in docs:
  n = n+1
print(n)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')